# Chronicling America API Assignment
In this assignment, you are tasked with:
* searching Chronicling America's API for a key word of your choice
* parsing your results from a dictionary to a `DataFrame` with headings "title", "city", 'date", and "raw_text"
* processing the raw text by removing "\n" characters, stopwords, and then lemmatizing the raw text before adding it to a new column called "lemmas."
* saving your `DataFrame` to a csv file

If you need any help with this assignment please email micah.saxton@tufts.edu


In [2]:
# imports
import pandas as pd
import requests
import json

In [ ]:
# initial search


In [ ]:
# find total amount of pages


In [ ]:
# query the api and save to dict 


In [ ]:
# convert dict to dataframe


In [ ]:
# convert date column from string to date-time object


In [ ]:
# sort by date


In [ ]:
# write fuction to process text


In [ ]:
# apply process_text function to raw_text column


In [ ]:
# save to csv
